In [1]:
import pandas as pd
import numpy as np

### Get data

In [2]:
dataset = pd.read_csv('train.csv')

dataset['Sex'] = dataset['Sex'].replace(['female', 'male'], [0, 1]) 
dataset['Embarked'] = dataset['Embarked'].replace(['S', 'C', 'Q'], [0, 1, 2]) 

dataset.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,0.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,1.0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0.0


### Split data in training and validation dataset

In [3]:
numeric_datapoints = dataset[['Pclass', 'Age', 'SibSp', 'Fare', 'Sex', 'Embarked']]
targets = dataset['Survived']

training_dataset_size = int(numeric_datapoints.shape[0] * 0.8)

training_points = numeric_datapoints.head(training_dataset_size)
training_labels = targets.head(training_dataset_size)

validation_dataset_size = int(numeric_datapoints.shape[0] - training_dataset_size)
validation_points = numeric_datapoints.tail(validation_dataset_size)
validation_labels = targets.tail(validation_dataset_size)

### Decision trees

In [4]:
import decision_trees

m_tree = decision_trees.ClassificationDecisionTree()
m_tree.train(training_points, training_labels, max_depth=3)

In [5]:
positives = 0

for i in range(training_dataset_size, numeric_datapoints.shape[0]):
    positives += (validation_labels.loc[i] == int(m_tree.predict(validation_points.loc[[i]])))
    
print('Accuracy : ', int(positives / (numeric_datapoints.shape[0] - training_dataset_size) * 100))

Accuracy :  86


In [6]:
test = pd.read_csv('test.csv')

test['Sex'] = test['Sex'].replace(['female', 'male'], [0, 1])
test['Embarked'] = test['Embarked'].replace(['S', 'C', 'Q'], [0, 1, 2]) 
test_important = test[['Pclass', 'Age', 'SibSp', 'Fare', 'Sex', 'Embarked']]

m_answer = pd.DataFrame()
for i in range(test.shape[0]):
    prediction = int(m_tree.predict(test_important.loc[[i]]))
    passenger_id = int(test.loc[i,'PassengerId'])
    m_answer = m_answer.append({'PassengerId':test.loc[i,'PassengerId'], 'Survived': prediction},ignore_index=True)
m_answer = m_answer.astype('int32')

In [7]:
m_answer.to_csv('titanic-dt.csv',index=False)